**POS-Tagging**

Welcome to the third lab! In this excercise you will build a simple pos-tagger.
The excercise is inspired from Pytorch tutorial site: https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [2]:
lstm = nn.LSTM(3, 3)  # Input dim is 3, output dim is 3
inputs = [torch.randn(1, 3) for _ in range(5)]  # make a sequence of length 5

# initialize the hidden state.
hidden = (torch.randn(1, 1, 3),
          torch.randn(1, 1, 3))
for i in inputs:
    # Step through the sequence one element at a time.
    # after each step, hidden contains the hidden state.
    out, hidden = lstm(i.view(1, 1, -1), hidden)

# alternatively, we can do the entire sequence all at once.
# the first value returned by LSTM is all of the hidden states throughout
# the sequence. the second is just the most recent hidden state
# (compare the last slice of "out" with "hidden" below, they are the same)
# The reason for this is that:
# "out" will give you access to all hidden states in the sequence
# "hidden" will allow you to continue the sequence and backpropagate,
# by passing it as an argument  to the lstm at a later time
# Add the extra 2nd dimension
inputs = torch.cat(inputs).view(len(inputs), 1, -1)
hidden = (torch.randn(1, 1, 3), torch.randn(1, 1, 3))  # clean out hidden state
out, hidden = lstm(inputs, hidden)





**Task:**

Load the `training_data` from `corpus-small.train`, and modify the `tag_to_ix` dictionary to have all different tags.
Build `ix_to_tag` dictionay, you will need it at the last task. 

In [3]:
def prepare_sequence(sent, to_ix):
    answer = []
    
    for word in sent:
        if word in to_ix:
            answer.append(to_ix[word])
        else:
            print(word)
            key, _ = random.choice(list(to_ix.items()))
            answer.append(to_ix[key])
    return  torch.tensor(answer)


training_data = []

train_file = open("corpus-small.train")
all_tags = set()

for line in train_file:
    sentence = line.split()
    words = [word.split("/")[0] for word in sentence]
    tags = [word.split("/")[1] for word in sentence]
    for word in sentence:
        all_tags.add(word.split("/")[1]) 
    training_data.append((words,tags))
    

word_to_ix = {}
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
#print(word_to_ix)
tag_to_ix = dict([(v,k)for k,v in enumerate(all_tags)])
ix_to_tag = dict([(v,k) for k,v in tag_to_ix.items()])
# print(training_data)
# print(tag_to_ix)
# print(ix_to_tag)

# These will usually be more like 32 or 64 dimensional.
# We will keep them small, so we can see how the weights change as we train.
EMBEDDING_DIM = 6
HIDDEN_DIM = 6

In [4]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [5]:
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# See what the scores are before training
# Note that element i,j of the output is the score for tag j for word i.
# Here we don't need to train, so the code is wrapped in torch.no_grad()
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
    print(tag_scores)

### Reduce number of epochs, if training data is big
for epoch in range(300):  # again, normally you would NOT do 300 epochs, it is toy data
    losses =[]
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)


        # Step 3. Run our forward pass.
        tag_scores = model(sentence_in)
       
        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(tag_scores, targets)
        losses.append(loss.item())
        loss.backward()
        optimizer.step()


# See what the scores are after training
import numpy as np
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
    # The sentence is "the dog ate the apple".  i,j corresponds to score for tag j
    # for word i. The predicted tag is the maximum scoring tag.
    # Here, we can see the predicted sequence below is 0 1 2 0 1
    # since 0 is index of the maximum value of row 1,
    # 1 is the index of maximum value of row 2, etc.
    # Which is DET NOUN VERB DET NOUN, the correct sequence!
    print(tag_scores)

tensor([[-3.4593, -4.0819, -3.4345,  ..., -3.9573, -3.3809, -3.7829],
        [-3.4245, -4.0567, -3.4257,  ..., -3.9175, -3.3929, -3.7357],
        [-3.3995, -4.0629, -3.5387,  ..., -3.9372, -3.4082, -3.7154],
        ...,
        [-3.4658, -4.0722, -3.4597,  ..., -4.0133, -3.4275, -3.6406],
        [-3.4989, -4.0778, -3.4268,  ..., -4.0245, -3.3863, -3.7695],
        [-3.3917, -3.9991, -3.4396,  ..., -3.9434, -3.3457, -3.8058]])
tensor([[ -3.5705,  -6.8395,  -6.6981,  ...,  -4.5009,  -6.7381,  -5.3725],
        [ -7.9810, -10.9088, -14.3828,  ...,  -5.1102, -11.7214, -16.8986],
        [ -5.2036,  -8.5442,  -6.9354,  ...,  -3.8986, -10.6075, -10.2440],
        ...,
        [ -5.1532,  -6.1926,  -4.5417,  ...,  -1.9362,  -7.1170,  -4.6931],
        [ -5.5997,  -9.9009,  -8.4219,  ...,  -3.0238, -11.4429, -10.9155],
        [-12.6629, -10.1369, -14.6736,  ..., -17.8635,  -8.1938,  -7.9987]])


**Task:**

Read the test data `corpus-small.test` and process it then get the predicitions.Write down the output tagged predicitons in file `corpus-small.out` in the same form as `corpus-small.answer`.

Note: in-case of unseen word in the testing dataset, replace it with a random seen one! (There's a better solution).

At the end, run the last cell, to get the accuracy of your model.

In [6]:
import numpy as np
import random


def idx_to_tag(idx):
    if  idx in ix_to_tag:
        return ix_to_tag[idx]
    else:
        return ix_to_tag[0]


test = open("corpus-small.test")
out = open("corpus-small.out","w")
sentences = []

for line in test:
    sentence = line.split()
    sentence_in = prepare_sequence(sentence,word_to_ix)
    tags =  np.argmax(model(sentence_in).data.numpy(),axis=1)
    #print(tags)
    answer = ""
    for i in range(len(sentence_in)):
        answer+=sentence[i]+"/"+str(idx_to_tag(tags[i]))+" "
    answer+="\n"
    out.write(answer)
test.close()
out.close()
#print(answer)


six
T.
Marshall
Hahn
Jr.
made
acquisitions
George
mode
kind
gentle
question
Can
act
hard-charging
Teddy
Roosevelt
Hahn
62-year-old
chairman
Georgia-Pacific
Corp.
leading
forest-product
unsolicited
3.19
bid
Great
Nekoosa
Corp
Nekoosa
given
public
cold
shoulder
reaction
Hahn
faced
18
earlier
acquisitions
negotiated
behind
scenes
So
far
Hahn
trying
entice
Nekoosa
negotiating
friendly
surrender
talking
tough
We
prepared
pursue
aggressively
completion
says
battle
opens
possibility
bidding
war
implies
If
competitor
enters
game
Hahn
face
dilemma
paying
premium
Nekoosa
seeing
fall
arms
rival
Given
associates
Hahn
industry
observers
former
university
developed
reputation
overpaying
anything
fold
Says
long-time
associate
Jerry
Griffin
WTD
Industries
school
winning
cost
manager
bid
Nekoosa
made
six
Georgia-Pacific
management
committee
signed
onto
deal
Hahn
knew
wanted
go
says
Correll
Associates
Hahn
picked
careful
management
Virginia
Polytechnic
Institute
Assuming
post
age
35
universities
says
Wa

In [7]:
%run tagger_eval.py corpus-small.out corpus-small.answer

Accuracy= 0.5652173913043478
